In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset
from scipy import sparse
import os
import copy
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader
from argoverse.map_representation.map_api import ArgoverseMap
from skimage.transform import rotate



/media/z/zMy/argoverse-api/map_files


In [2]:
config = dict()
config['data_dir'] = dict()
config['data_dir']['train'] = 'dataset/train/data'
config['pred_range'] = 40 

In [4]:
class ArgoDataset(Dataset):
    def __init__(self, split, config):
        self.config = config
        self.avl = ArgoverseForecastingLoader(config.data_dir[split])
        self.avl.seq_list = sorted(self.avl.seq_list)
        self.am = ArgoverseMap()
            
    def __getitem__(self, idx):
        data = self.read_argo_data(idx)
        data = self.get_obj_feats(data)
        data['idx'] = idx
        data['graph'] = self.get_lane_graph(data)
        return data
    
    def __len__(self):
        if 'preprocess' in self.config and self.config['preprocess']:
            return len(self.split)
        else:
            return len(self.avl)

    def get_center

    def read_argo_data(self, idx):
        city = copy.deepcopy(self.avl[idx].city)

        """TIMESTAMP,TRACK_ID,OBJECT_TYPE,X,Y,CITY_NAME"""
        df = copy.deepcopy(self.avl[idx].seq_df)
        
        agt_ts = np.sort(np.unique(df['TIMESTAMP'].values))
        mapping = dict()
        for i, ts in enumerate(agt_ts):
            mapping[ts] = i

        trajs = np.concatenate((
            df.X.to_numpy().reshape(-1, 1),
            df.Y.to_numpy().reshape(-1, 1)), 1)
        
        steps = [mapping[x] for x in df['TIMESTAMP'].values]
        steps = np.asarray(steps, np.int64)

        objs = df.groupby(['TRACK_ID', 'OBJECT_TYPE']).groups
        keys = list(objs.keys())
        obj_type = [x[1] for x in keys]

        agt_idx = obj_type.index('AGENT')
        idcs = objs[keys[agt_idx]]
       
        agt_traj = trajs[idcs]
        agt_step = steps[idcs]

        del keys[agt_idx]
        ctx_trajs, ctx_steps = [], []
        for key in keys:
            idcs = objs[key]
            ctx_trajs.append(trajs[idcs])
            ctx_steps.append(steps[idcs])

        data = dict()
        data['city'] = city
        data['trajs'] = [agt_traj] + ctx_trajs
        data['steps'] = [agt_step] + ctx_steps
        return data
    
    def get_obj_feats(self, data):
        orig = data['trajs'][0][19].copy().astype(np.float32)

        if self.train and self.config['rot_aug']:
            theta = np.random.rand() * np.pi * 2.0
        else:
            pre = data['trajs'][0][18] - orig
            theta = np.pi - np.arctan2(pre[1], pre[0])

        rot = np.asarray([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]], np.float32)

        feats, ctrs, gt_preds, has_preds = [], [], [], []
        for traj, step in zip(data['trajs'], data['steps']):
            if 19 not in step:
                continue

            gt_pred = np.zeros((30, 2), np.float32)
            has_pred = np.zeros(30, np.bool)
            future_mask = np.logical_and(step >= 20, step < 50)
            post_step = step[future_mask] - 20
            post_traj = traj[future_mask]
            gt_pred[post_step] = post_traj
            has_pred[post_step] = 1
            
            obs_mask = step < 20
            step = step[obs_mask]
            traj = traj[obs_mask]
            idcs = step.argsort()
            step = step[idcs]
            traj = traj[idcs]
            
            for i in range(len(step)):
                if step[i] == 19 - (len(step) - 1) + i:
                    break
            step = step[i:]
            traj = traj[i:]

            feat = np.zeros((20, 3), np.float32)
            feat[step, :2] = np.matmul(rot, (traj - orig.reshape(-1, 2)).T).T
            feat[step, 2] = 1.0

            x_min, x_max, y_min, y_max = self.config['pred_range']
            if feat[-1, 0] < x_min or feat[-1, 0] > x_max or feat[-1, 1] < y_min or feat[-1, 1] > y_max:
                continue

            ctrs.append(feat[-1, :2].copy())
            feat[1:, :2] -= feat[:-1, :2]
            feat[step[0], :2] = 0
            feats.append(feat)
            gt_preds.append(gt_pred)
            has_preds.append(has_pred)

        feats = np.asarray(feats, np.float32)
        ctrs = np.asarray(ctrs, np.float32)
        gt_preds = np.asarray(gt_preds, np.float32)
        has_preds = np.asarray(has_preds, np.bool)

        data['feats'] = feats
        data['ctrs'] = ctrs
        data['orig'] = orig
        data['theta'] = theta
        data['rot'] = rot
        data['gt_preds'] = gt_preds
        data['has_preds'] = has_preds
        return data

 
    def get_lane_graph(self, data):
        """Get a rectangle area defined by pred_range."""
        x_min, x_max, y_min, y_max = self.config['pred_range']
        radius = max(abs(x_min), abs(x_max)) + max(abs(y_min), abs(y_max))
        lane_ids = self.am.get_lane_ids_in_xy_bbox(data['orig'][0], data['orig'][1], data['city'], radius)
        lane_ids = copy.deepcopy(lane_ids)
        
     

a = ArgoDataset('train',config)

SyntaxError: invalid syntax (719328813.py, line 21)

In [5]:
split='train'
avl = ArgoverseForecastingLoader(config['data_dir'][split])
avl.seq_list = sorted(avl.seq_list)
am = ArgoverseMap()

In [6]:
import json
config_file = open('config.json')
config = json.load(config_file)

In [7]:
def contain_pt(pt, size):
    return pt[0] >= 0 and pt[0] < size \
            and pt[0] >= 0 and pt[1] < size

In [ ]:
d

In [8]:
l = config['pred_range']
gap = config['image_pixel_size']
img_sz = len(np.arange(-l,l,gap))
img = np.zeros((img_sz,img_sz,45),dtype='float64')

id = 0
df = copy.deepcopy(avl[0].seq_df)

timestamp = np.sort(np.unique(df['TIMESTAMP'].values))

agent_traj = df[df['OBJECT_TYPE']=='AGENT'][['X','Y']].values

center = agent_traj[-1,:]

c = 0
for i, ts in enumerate(timestamp):
    df_i = df[df['TIMESTAMP']==ts]
    for _, row in df_i.iterrows():
        position = row[['X','Y']].values
        obj_type = row['OBJECT_TYPE']
        img_position = np.rint(((position - center + l) / gap).astype('float64')) \
        .astype('int').tolist()
        
        if contain_pt(img_position, img_sz):
            channel = i if obj_type == 'AGENT' else i + 20
            img[tuple(img_position)][0] += 1
            c += 1
    #time_map[ts] = i


#print(df[df['OBJECT_TYPE']=='OTHERS'])32


#for i in range(20):


In [53]:
from shapely.geometry.polygon import Polygon, Point
from operator import or_
from functools import reduce
import cv2
l = 56
#driveable area

lane_ids = am.get_lane_ids_in_xy_bbox(center[0], center[1], 'PIT', 50)
polygons = am.find_local_driveable_areas([center[0]-l,center[0]+l,center[1]-l,center[1]+l],'PIT')

img_driveable_area = np.zeros((img_sz,img_sz),dtype='float64')
img_lane_boundaries = np.zeros((img_sz,img_sz),dtype='float64')
img_centerline = np.zeros((3,img_sz,img_sz),dtype='float64')

for i in polygons:
    for point in i:
        img_position = np.rint(((point[0:2] - center + l) / gap).astype('float64')) \
        .astype('int').tolist()
        #img_driveable_area[] += 1
        if contain_pt(img_position, img_sz):
            img_driveable_area[tuple(img_position)] = 1

'''
import queue
que = queue.Queue()
center_img_position = np.rint(np.array([l,l]) / gap).astype('int')
que.put(center_img_position)
img_driveable_area[tuple(center_img_position)] = 255
deltas = np.array([[1,0],[-1,0],[0,1],[0,-1]])

while not que.empty():
    img_position = que.get()
    for delta in deltas:
        new_img_position = img_position + delta
        #print(img_driveable_area[new_img_position])
        if contain_pt(new_img_position, img_sz) and \
            img_driveable_area[tuple(new_img_position)] == 0:
            que.put(new_img_position)
            #print(new_img_position)
            img_driveable_area[tuple(new_img_position)] = 255


'''
#centerline and lane_boundaries
for lane_id in lane_ids:
    lane = am.city_lane_centerlines_dict['PIT'][lane_id]
    #print(lane.centerline)
    centerline = lane.centerline
    vector = centerline[-1] - centerline[0]
    line_mid = (centerline[-1] + centerline[0]) / 2
    
    polygon = am.get_lane_segment_polygon(lane_id, 'PIT')
    for pt in polygon:
        img_position = np.rint(((pt[0:2] - center + l) / gap).astype('float64')) \
        .astype('int').tolist()
        if contain_pt(img_position, img_sz):
            img_lane_boundaries[tuple(img_position)] = 255
    
    line_mid_img_position = np.rint(((line_mid[0:2] - center + l) / gap).astype('float64')) \
        .astype('int').tolist()

    turn_direction = 0
    if lane.turn_direction == 'LEFT':
        turn_direction = 0.5
    elif lane.turn_direction == 'RIGHT':
        turn_direction = 1
    img_centerline[0,tuple(line_mid_img_position)] = turn_direction
    

    img_centerline[1:3][:,line_mid_img_position[0],line_mid_img_position[1]] = vector / np.linalg.norm(vector)
    #print(img_centerline[1:3][:,line_mid_img_position[0],line_mid_img_position[1]].shape)
    break
    #print(vector/np.linalg.norm(vector))
#t

In [27]:
a = np.zeros((2,2))
a[(1,1)] = 1
a

array([[0., 0.],
       [0., 1.]])

In [64]:
from torch import nn
import torch
n = nn.Softmax()
a = torch.randn((2,3))
print(n(a))

tensor([[[1., 1., 1.],
         [1., 1., 1.]]])


/tmp/ipykernel_133217/4030804543.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(n(a))
